## Second Assignment

### Content

### Requirements

In [11]:
import os
import sys
import spacy
import pandas as pd

In [12]:
sys.path.insert(0, os.path.abspath('src/'))
from conll import read_corpus_conll, evaluate
from spacy.tokens import Doc
conll2003 = read_corpus_conll('data/conll2003/test.txt')

### Task 1

- ### Task 1.1

In [13]:
# utility fns
def decode_ref(conll_tuple: tuple):
    return (get_token(conll_tuple), get_iob(conll_tuple))  # inner text of token + iob tag

def get_token(conll_tuple: tuple):
    decoded = conll_tuple[0].strip().split()
    return decoded[0]

def get_iob(conll_tuple: tuple):
    decoded = conll_tuple[0].strip().split()
    return decoded[-1]

def encode_key(lst: list):
    return "[{}]".format(", ".join(sorted(lst)))

In [14]:
def token_acc(refs, hyps):
    # initialize the dictionary to be use for token accuracy on a class level.
    # each key will represent the name of the class, and will contain:
    #   - 'cor': correctly classified samples of that class
    #   - 'tot': overall number of examples of that class
    ret = {}  
    for sent_ref, sent_hyp in zip(refs, hyps):
        for (ref_text,ref_iob), (hyp_text, hyp_iob) in zip(sent_ref, sent_hyp):
            
            # create the class-based dictionary with default data if needed
            if ref_iob not in ret.keys():
                ret[ref_iob] = {'cor': 0, 'tot': 0}
            
            if hyp_iob not in ret.keys():
                ret[hyp_iob] = {'cor': 0, 'tot': 0}
            
            # update counts of both total and correct
            ret[ref_iob]['tot'] += 1
            if ref_iob == hyp_iob:
                ret[hyp_iob]['cor'] += 1
        
    # compute class based accuracy and aggregate
    total_correct = 0
    total = 0
    for key, sub_dict in ret.items():
        sub_dict['acc'] = sub_dict['cor'] / sub_dict['tot']
        total_correct += sub_dict['cor']
        total += sub_dict['tot']

    ret['aggregate'] = {'cor': total_correct, 'tot': total, 'acc': total_correct / total}
    return ret

In [15]:
# defining a custom whitespace tokenizer
class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        super(WhitespaceTokenizer, self).__init__()
        self.vocab = vocab

    def __call__(self, text):
        words = text.split(" ")
        spaces = [True] * len(words); spaces[-1] = False  # last word of a sentence doesn't need trailing space
        return Doc(self.vocab, words=words, spaces=spaces)
    
# initializing the default pipeline and substituting the tokenizer.
# this allows tokens provided by spacy and the original tokens from the conll2003 dataset to match.
nlp = spacy.load("en_core_web_sm")
nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

In [16]:
# Task 0: evaluate spaCy NER on CoNLL 2003 data
#  - report token-level performance
#    - accuracy of correctly recognizing all tokens that belong to named entities
#  - report CoNLL chunk-level performance (per class and total);
#    - precision, recall, f-measure of correctly recognizing all the named entities in a chunk per class and total

# build the refs (tuples of token + iob tag wrt to NER task)
refs, hyps = [], []
for sent in conll2003:  # discard first line, DOCSTART
    # feed spaCy Doc with pre-defined tokens and run the NER
    tokens = [get_token(tpl) for tpl in sent]
    if " ".join(tokens).startswith("-DOCSTART-"): continue
    doc = nlp(" ".join(tokens))
        
    # we must now convert spaCy NE labels
    iobs_pred = []
    for t in doc:
        if t.ent_iob_ != 'O':
            if t.ent_type_ in ['ORG', 'LOC']:
                iobs_pred.append((t.text, f"{t.ent_iob_}-{t.ent_type_}"))
            elif t.ent_type_ == 'PERSON':
                iobs_pred.append((t.text, f"{t.ent_iob_}-PER"))
            else:
                iobs_pred.append((t.text, f"{t.ent_iob_}-MISC"))
        else:
            iobs_pred.append((t.text, 'O'))

    # then append data for evaluation
    refs.append([decode_ref(tpl) for tpl in sent])
    hyps.append(iobs_pred)
    
# to get the token-level accuracy, let's use our custom fn
acc = token_acc(refs, hyps)

# to get class-level scores, let's make use of conll.py
results = evaluate(refs, hyps)

In [17]:
pd.DataFrame().from_dict(acc, orient='index').round(decimals=3)

,cor,tot,acc
O,33041,38323,0.862
B-LOC,28,1668,0.017
B-PER,985,1617,0.609
B-MISC,410,702,0.584
B-ORG,504,1661,0.303
I-PER,874,1156,0.756
I-ORG,434,835,0.520
I-LOC,20,257,0.078
I-MISC,88,216,0.407
aggregate,36384,46435,0.784


- ### Task 1.2

In [18]:
pd.DataFrame().from_dict(results, orient='index').round(decimals=3)

,p,r,f,s
LOC,0.481,0.016,0.030,1668
PER,0.761,0.590,0.665,1617
MISC,0.078,0.567,0.137,702
ORG,0.448,0.272,0.339,1661
total,0.246,0.324,0.280,5648


### Task 2

In [19]:
# Task 2
# Grouping of Entities.

# Write a function to group recognized named entities using noun_chunks method of spaCy.
def group_sent(doc: Doc):
    stored_ents = []
    ent_groups = []

    # add all groups that appear in noun chunks
    for nc in doc.noun_chunks:
        to_store = []
        for ent in nc.ents:
            if ent not in stored_ents:
                to_store.append(ent.label_)
                stored_ents.append(ent)
        if len(to_store) > 0:
            ent_groups.append(sorted(to_store))          
    
    # take also into account entities that do not appear in noun chunks
    # and count them as groups composed by only 1 NE
    for ent in doc.ents:
        if ent not in stored_ents:
            ent_groups.append([ent.label_])
    
    return ent_groups


nlp = spacy.load("en_core_web_sm")
groups = []

for sent in conll2003:
    # rebuild the original sentence and process it
    tokens = [get_token(tpl) for tpl in sent]
    # skip useless lines
    if " ".join(tokens).startswith("-DOCSTART-"): continue
    # otherwise, process the doc to be fed into the group fn
    doc = nlp(" ".join(tokens))
    # store the overall output
    sent_groups = group_sent(doc)
    groups.append(sent_groups)
    

# Analyze the groups in terms of most frequent combinations (i.e. NER types that go together)
def fl(groups):
    fdict = {}
    # scan each sentence
    for sent_groups in groups:
        # scan each group within a sentence
        for group in sent_groups:  # ATTENTION: 'g' is a list and can be empty
            if len(group) == 0: continue
            key = encode_key(group)
            # create the dict with default counts
            if key not in fdict.keys():
                fdict[key] = 0
            # update the counts for the given group
            fdict[key] += 1
    return fdict

out = dict(sorted(fl(groups).items(), key=lambda item: item[1], reverse=True))
[print(k,v) for k,v in out.items()]

[CARDINAL] 1624
[GPE] 1255
[PERSON] 1074
[DATE] 997
[ORG] 873
[NORP] 293
[MONEY] 147
[ORDINAL] 111
[TIME] 83
[PERCENT] 81
[EVENT] 58
[LOC] 54
[CARDINAL, PERSON] 53
[QUANTITY] 51
[NORP, PERSON] 44
[GPE, PERSON] 38
[GPE, GPE] 26
[FAC] 22
[PRODUCT] 22
[ORG, PERSON] 22
[CARDINAL, ORG] 22
[GPE, ORG] 18
[CARDINAL, GPE] 17
[CARDINAL, NORP] 15
[DATE, ORG] 14
[NORP, ORG] 12
[LAW] 11
[WORK_OF_ART] 10
[GPE, PRODUCT] 10
[DATE, EVENT] 8
[ORG, ORG] 8
[PERSON, PERSON] 8
[DATE, TIME] 7
[NORP, ORDINAL] 6
[LANGUAGE] 6
[DATE, NORP] 6
[DATE, GPE] 5
[CARDINAL, CARDINAL, PERSON] 5
[GPE, ORDINAL] 5
[CARDINAL, DATE] 5
[CARDINAL, CARDINAL] 5
[ORDINAL, PERSON] 5
[ORDINAL, ORG] 3
[DATE, NORP, PERSON] 3
[EVENT, ORDINAL] 3
[NORP, NORP] 3
[FAC, GPE] 3
[CARDINAL, EVENT] 3
[CARDINAL, GPE, PERSON] 2
[PERSON, PERSON, PERSON] 2
[CARDINAL, ORDINAL] 2
[LANGUAGE, ORDINAL] 2
[CARDINAL, PERCENT] 2
[DATE, GPE, ORG] 2
[GPE, LOC] 2
[CARDINAL, CARDINAL, ORG] 2
[QUANTITY, QUANTITY] 2
[GPE, NORP] 2
[DATE, ORDINAL] 1
[GPE, ORDINAL,

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

### Task 3

In [21]:
# One of the possible post-processing steps is to fix segmentation errors.
# Write a function that extends the entity span to cover the full noun-compounds.
# Make use of 'compound' dependency relation

def extend_entity_span(ent: spacy.tokens.span.Span):
    ent_ext = []
    # checking noun compounds for each token
    for t in ent:
        # appending the current token to the extended entity
        ent_ext.append(t)
        # if the token is a compound wrt sth outside of the entity, let's add the head too
        if t.dep_ == "compound" and t.head not in ent and t.head not in ent_ext:
            ent_ext.append(t.head)
            
    # save everything
    ent_ext = sorted(ent_ext, key=lambda x:x.i)
    ent_ext = ent.doc[ent_ext[0].i:ent_ext[-1].i+1]  # as a Span object
    return ent_ext

def filter_ents(ents: list):
    # now, discard all the extended entities that are fully included into others
    extended_filtered = []
    for ext_ent in ents:
        # perform type check for each item of the provided list
        assert type(ext_ent) == spacy.tokens.span.Span, \
        "(fn: filter_ents) each item of argument 'ents' must be a Span obj."

        keep = True
        start, stop = ext_ent[0].i, ext_ent[-1].i
        for ext_match in ents:
            match_start, match_stop = ext_match[0].i, ext_match[-1].i       
            # now, check if the span intersection is not empty
            # in order, the following types of inclusion are checked:
            # - strict inclusion;
            # - non-strict left-inclusion;
            # - non-strict right-inclusion.
            if (match_start < start and stop < match_stop) or \
            (match_start <= start and stop < match_stop) or \
            (match_start < start and stop <= match_stop): 
                keep = False
                break
        
        if keep: extended_filtered.append(ext_ent)
    return extended_filtered

# test out the implementation on the conll2003 dataset
for sent in conll2003:
    tokens = [get_token(tpl) for tpl in sent]
    if " ".join(tokens).startswith("-DOCSTART-"): continue
    doc = nlp(" ".join(tokens))
    original = [ent for ent in doc.ents]
    extended = [extend_entity_span(ent) for ent in doc.ents]
    filtered = filter_ents(extended)
    print("'{}'".format(" ".join(tokens)))
    print("Original entities: ", original)
    print("Extended entities: ", extended)
    print("Filtered entities: ", filtered)
    print("\n\n")

'SOCCER - JAPAN GET LUCKY WIN , CHINA IN SURPRISE DEFEAT .'
Original entities:  [CHINA]
Extended entities:  [CHINA]
Filtered entities:  [CHINA]



'Nadim Ladki'
Original entities:  [Nadim Ladki]
Extended entities:  [Nadim Ladki]
Filtered entities:  [Nadim Ladki]



'AL-AIN , United Arab Emirates 1996-12-06'
Original entities:  [AL-AIN, United Arab Emirates, 1996-12-06]
Extended entities:  [AL-AIN, United Arab Emirates, 1996-12-06]
Filtered entities:  [AL-AIN, United Arab Emirates, 1996-12-06]



'Japan began the defence of their Asian Cup title with a lucky 2-1 win against Syria in a Group C championship match on Friday .'
Original entities:  [Japan, their Asian Cup, Syria, Group C, Friday]
Extended entities:  [Japan, their Asian Cup title, Syria, Group C championship, Friday]
Filtered entities:  [Japan, their Asian Cup title, Syria, Group C championship, Friday]



'But China saw their luck desert them in the second match of the group , crashing to a surprise 2-0 defeat to newcomers U

'I think now is the right time for him to return . "'
Original entities:  []
Extended entities:  []
Filtered entities:  []



'Squad : Javier Pertile , Paolo Vaccari , Marcello Cuttitta , Ivan Francescato , Leandro Manteri , Diego Dominguez , Francesco Mazzariol , Alessandro Troncon , Orazio Arancio , Andrea Sgorlon , Massimo Giovanelli , Carlo Checchinato , Walter Cristofoletto , Franco Properzi Curti , Carlo Orlandi , Massimo Cuttitta , Giambatista Croci , Gianluca Guidi , Nicola Mazzucato , Alessandro Moscardi , Andrea Castellani .'
Original entities:  [Javier Pertile, Paolo Vaccari, Marcello Cuttitta, Ivan Francescato, Leandro Manteri, Diego Dominguez, Francesco Mazzariol, Alessandro Troncon, Orazio Arancio, Andrea Sgorlon, Massimo Giovanelli, Carlo Checchinato, Walter Cristofoletto, Franco Properzi Curti, Carlo Orlandi, Massimo Cuttitta, Giambatista Croci, Gianluca Guidi, Nicola Mazzucato, Alessandro Moscardi, Andrea Castellani]
Extended entities:  [Javier Pertile, Paolo Vaccari, 

'7. Jim Moran ( U.S. ) 23.25'
Original entities:  [7, Jim Moran, U.S., 23.25]
Extended entities:  [7, Jim Moran, U.S., 23.25]
Filtered entities:  [7, Jim Moran, U.S., 23.25]



'8. Dominick Gauthier ( Canada ) 22.73'
Original entities:  [8, Dominick Gauthier, 22.73]
Extended entities:  [8, Dominick Gauthier, 22.73]
Filtered entities:  [8, Dominick Gauthier, 22.73]



'9. Johann Gregoire ( France ) 22.58'
Original entities:  [9, Johann Gregoire, France, 22.58]
Extended entities:  [9, Johann Gregoire, France, 22.58]
Filtered entities:  [9, Johann Gregoire, France, 22.58]



'10. Troy Benson ( U.S. ) 22.56'
Original entities:  [10, Troy Benson, U.S., 22.56]
Extended entities:  [10, Troy Benson, U.S., 22.56]
Filtered entities:  [10, Troy Benson, U.S., 22.56]



'Women'
Original entities:  []
Extended entities:  []
Filtered entities:  []



'1. Tatjana Mittermayer ( Germany ) 24.32'
Original entities:  [1, Tatjana Mittermayer, Germany, 24.32]
Extended entities:  [1, Tatjana Mittermayer, Ger

'Bowling : Doull 8-1-60-0 ( w-3 ) , Kennedy 3-0-24-0 ( w-7 nb-1 ) ,'
Original entities:  [8, Kennedy, 3]
Extended entities:  [8, Kennedy, 3]
Filtered entities:  [8, Kennedy, 3]



'Cairns 8-1-35-1 ( w-2 ) , Vaughan 9-1-55-1 , Harris 10-0-42-5 ( w-1 ) ,'
Original entities:  [8, 9, Harris, 10]
Extended entities:  [8, 9, Harris, 10]
Filtered entities:  [8, 9, Harris, 10]



'Astle 9-0-53-1 ( w-1 nb-1 )'
Original entities:  []
Extended entities:  []
Filtered entities:  []



'New Zealand innings'
Original entities:  [New Zealand]
Extended entities:  [New Zealand innings]
Filtered entities:  [New Zealand innings]



'B. Young c Moin Khan b Waqar 5'
Original entities:  [B. Young, Moin Khan]
Extended entities:  [B. Young, Moin Khan]
Filtered entities:  [B. Young, Moin Khan]



'C. Spearman c Moin Khan b Wasim 0'
Original entities:  [C. Spearman, Moin Khan b Wasim]
Extended entities:  [C. Spearman, Moin Khan b Wasim]
Filtered entities:  [C. Spearman, Moin Khan b Wasim]



'A. Parore c Ijaz Ahm

'CSKA Moscow ( Russia 9 6 3 15'
Original entities:  [Moscow, Russia]
Extended entities:  [Moscow, Russia]
Filtered entities:  [Moscow, Russia]



'Stefanel Milan ( Italy ) 9 6 3 15'
Original entities:  [Stefanel Milan, Italy, 9]
Extended entities:  [Stefanel Milan, Italy, 9]
Filtered entities:  [Stefanel Milan, Italy, 9]



'Maccabi Tel Aviv ( Israel ) 9 5 4 14'
Original entities:  [Maccabi Tel Aviv, Israel, 9]
Extended entities:  [Maccabi Tel Aviv, Israel, 9]
Filtered entities:  [Maccabi Tel Aviv, Israel, 9]



'Ulker Spor ( Turkey ) 9 4 5 13'
Original entities:  [Ulker Spor, Turkey, 9]
Extended entities:  [Ulker Spor, Turkey, 9]
Filtered entities:  [Ulker Spor, Turkey, 9]



'Limoges ( France ) 9 3 6 12'
Original entities:  [France]
Extended entities:  [France]
Filtered entities:  [France]



'Panionios ( Greece ) 9 3 6 12'
Original entities:  [Greece, 9 3 6 12]
Extended entities:  [Greece, 9 3 6 12]
Filtered entities:  [Greece, 9 3 6 12]



'Group B'
Original entities:  []
Extended 

...

'The pair hammered 36 boundaries between them .'
Original entities:  [36]
Extended entities:  [36]
Filtered entities:  [36]



'Pace bowler Ian Harvey claimed three for 81 for Victoria .'
Original entities:  [Ian Harvey, three, 81, Victoria]
Extended entities:  [Ian Harvey, three, 81, Victoria]
Filtered entities:  [Ian Harvey, three, 81, Victoria]



'CRICKET - SHEFFIELD SHIELD SCORE .'
Original entities:  []
Extended entities:  []
Filtered entities:  []



'HOBART , Australia 1996-12-07'
Original entities:  [HOBART]
Extended entities:  [HOBART]
Filtered entities:  [HOBART]



'Close of play score on the second day of the four-day Sheffield Shield cricket match between Tasmania and Victoria at Bellerive Oval on Saturday :'
Original entities:  [the second day, four-day, Sheffield Shield, Tasmania and Victoria, Bellerive Oval, Saturday]
Extended entities:  [the second day, four-day Sheffield Shield cricket match, Sheffield Shield cricket match, Tasmania and Victoria, Bellerive Oval, Satu

Original entities:  []
Extended entities:  []
Filtered entities:  []



'Hapoel Beit She'an 2 Hapoel Beersheba 1'
Original entities:  [Hapoel Beit She'an, 2, Hapoel Beersheba]
Extended entities:  [Hapoel Beit She'an, 2, Hapoel Beersheba]
Filtered entities:  [Hapoel Beit She'an, 2, Hapoel Beersheba]



'Maccabi Petah Tikva 0 Hapoel Haifa 2'
Original entities:  [Hapoel Haifa]
Extended entities:  [Hapoel Haifa]
Filtered entities:  [Hapoel Haifa]



'Standings ( tabulate under played , won , drawn , lost , goals'
Original entities:  []
Extended entities:  []
Filtered entities:  []



'for , against , points ) :'
Original entities:  []
Extended entities:  []
Filtered entities:  []



'Betar Jerusalem 12 10 2 0 28 7 32'
Original entities:  [Betar Jerusalem, 12 10 2]
Extended entities:  [Betar Jerusalem, 12 10 2]
Filtered entities:  [Betar Jerusalem, 12 10 2]



'Hapoel Petah Tikva 12 9 2 1 27 13 29'
Original entities:  [Hapoel Petah Tikva]
Extended entities:  [Hapoel Petah Tikva]
Filtered en

'CHICAGO 17 1 .944 -'
Original entities:  []
Extended entities:  []
Filtered entities:  []



'DETROIT 14 3 .824 2 1/2'
Original entities:  [DETROIT, 14 3, 2 1/2]
Extended entities:  [DETROIT, 14 3, 2 1/2]
Filtered entities:  [DETROIT, 14 3, 2 1/2]



'CLEVELAND 11 6 .647 5 1/2'
Original entities:  [11 6, 5 1/2]
Extended entities:  [11 6, 5 1/2]
Filtered entities:  [11 6, 5 1/2]



'ATLANTA 10 8 .556 7'
Original entities:  [ATLANTA, 8]
Extended entities:  [ATLANTA, 8]
Filtered entities:  [ATLANTA, 8]



'MILWAUKEE 8 8 .500 8'
Original entities:  [8 8 .500]
Extended entities:  [8 8 .500]
Filtered entities:  [8 8 .500]



'INDIANA 8 8 .500 8'
Original entities:  [8 8 .500]
Extended entities:  [8 8 .500]
Filtered entities:  [8 8 .500]



'CHARLOTTE 8 9 .471 8 1/2'
Original entities:  [8 1/2]
Extended entities:  [8 1/2]
Filtered entities:  [8 1/2]



'TORONTO 6 11 .353 10 1/2'
Original entities:  [6 11, 10 1/2]
Extended entities:  [6 11, 10 1/2]
Filtered entities:  [6 11, 10 1/2]



'WESTE

'TAMPA BAY 8 15 2 69 81 18'
Original entities:  [8 15 2 69 81 18]
Extended entities:  [8 15 2 69 81 18]
Filtered entities:  [8 15 2 69 81 18]



'WESTERN CONFERENCE'
Original entities:  []
Extended entities:  []
Filtered entities:  []



'CENTRAL DIVISION'
Original entities:  []
Extended entities:  []
Filtered entities:  []



'W L T GF GA PTS'
Original entities:  [W L T GF GA PTS]
Extended entities:  [W L T GF GA PTS]
Filtered entities:  [W L T GF GA PTS]



'DETROIT 15 9 4 81 53 34'
Original entities:  [DETROIT]
Extended entities:  [DETROIT]
Filtered entities:  [DETROIT]



'DALLAS 16 10 1 77 66 33'
Original entities:  [16]
Extended entities:  [16]
Filtered entities:  [16]



'ST LOUIS 14 14 0 82 84 28'
Original entities:  [ST]
Extended entities:  [ST LOUIS]
Filtered entities:  [ST LOUIS]



'CHICAGO 12 13 3 72 70 27'
Original entities:  []
Extended entities:  []
Filtered entities:  []



'TORONTO 11 16 0 81 95 22'
Original entities:  [11]
Extended entities:  [11]
Filtered entities: 

'A mix-up in the Barcelona defence let Croatian international Suker in midway through the first half , and Montenegrin striker Mijatovic made it 2-0 after fine work by Clarence Seedorf just after the break .'
Original entities:  [Barcelona, Croatian, the first half, Montenegrin, Mijatovic, 2, Clarence Seedorf]
Extended entities:  [Barcelona defence, Croatian, the first half, Montenegrin striker, Mijatovic, 2, Clarence Seedorf]
Filtered entities:  [Barcelona defence, Croatian, the first half, Montenegrin striker, Mijatovic, 2, Clarence Seedorf]



'Barcelona fought back strongly but were twice denied by the woodwork on an unusually quiet night for Brazilian striker Ronaldo .'
Original entities:  [night, Brazilian, Ronaldo]
Extended entities:  [night, Brazilian, Ronaldo]
Filtered entities:  [night, Brazilian, Ronaldo]



'SOCCER - PSV HIT VOLENDAM FOR SIX .'
Original entities:  []
Extended entities:  []
Filtered entities:  []



'AMSTERDAM 1996-12-07'
Original entities:  []
Extended enti